In [28]:
from PIL import Image
import numpy as np

def diffuse_image(image_path, diffusion_coefficient, time_step):
    # 打开图像并将其转换为灰度图像
    image = Image.open(image_path).convert('L')
    # 将图像转换为numpy数组
    image_array = np.array(image)
    # 获取图像的形状
    shape = image_array.shape
    # 创建一个新的数组来存储扩散后的图像
    diffused_image = np.zeros(shape)
    # 遍历图像中的每个像素
    for i in range(shape[0]):
        for j in range(shape[1]):
            # 计算当前像素周围的像素值之和
            sum_of_neighbors = 0
            if i > 0:
                sum_of_neighbors += image_array[i-1, j]
            if i < shape[0]-1:
                sum_of_neighbors += image_array[i+1, j]
            if j > 0:
                sum_of_neighbors += image_array[i, j-1]
            if j < shape[1]-1:
                sum_of_neighbors += image_array[i, j+1]
            # 计算当前像素的扩散值
            diffused_value = image_array[i, j] + diffusion_coefficient * time_step * (sum_of_neighbors - 4 * image_array[i, j])
            # 将扩散值存储在新数组中
            diffused_image[i, j] = diffused_value
    # 将新数组转换为图像并返回
    diffused_image = np.clip(diffused_image, 0, 255).astype(np.uint8)
    return Image.fromarray(diffused_image, 'L')

In [66]:
diffused_image = diffuse_image('test/nii2png/8.png', 0.25, 10)
diffused_image.show()
diffused_image.save('test/nii2png/8_diffused.png')

In [56]:
from PIL import Image
import numpy as np

def denoise_image(image_path, diffusion_coefficient, time_step, iterations):
    # 打开图像并将其转换为灰度图像
    image = Image.open(image_path).convert('L')
    # 将图像转换为numpy数组
    image_array = np.array(image)
    # 获取图像的形状
    shape = image_array.shape
    # 创建一个新的数组来存储去噪后的图像
    denoised_image = np.copy(image_array)
    # 迭代进行去噪处理
    for _ in range(iterations):
        # 遍历图像中的每个像素
        for i in range(shape[0]):
            for j in range(shape[1]):
                # 计算当前像素周围的像素值之和
                sum_of_neighbors = 0
                if i > 0:
                    sum_of_neighbors += denoised_image[i-1, j]
                if i < shape[0]-1:
                    sum_of_neighbors += denoised_image[i+1, j]
                if j > 0:
                    sum_of_neighbors += denoised_image[i, j-1]
                if j < shape[1]-1:
                    sum_of_neighbors += denoised_image[i, j+1]
                # 计算当前像素的扩散值
                diffused_value = denoised_image[i, j] + diffusion_coefficient * time_step * (sum_of_neighbors - 4 * denoised_image[i, j])
                # 将扩散值存储在新数组中
                denoised_image[i, j] = diffused_value
    # 将新数组转换为图像并返回
    return Image.fromarray(denoised_image)

In [76]:
denoised_image = denoise_image('test/nii2png/8.png', 0.25, 1, 5)
denoised_image.show()
denoised_image.save('test/nii2png/8_denoised.png')

In [42]:
def nonlinear_diffusion(image_array, diffusion_coefficient, time_step, kappa):
    shape = image_array.shape
    diffused_image = np.copy(image_array)

    for i in range(shape[0]):
        for j in range(shape[1]):
            sum_of_neighbors = 0

            if i > 0:
                grad_n = abs(image_array[i, j] - image_array[i-1, j])
                c_n = np.exp(-(grad_n/kappa)**2)
                sum_of_neighbors += c_n * image_array[i-1, j]

            if i < shape[0]-1:
                grad_s = abs(image_array[i, j] - image_array[i+1, j])
                c_s = np.exp(-(grad_s/kappa)**2)
                sum_of_neighbors += c_s * image_array[i+1, j]

            if j > 0:
                grad_w = abs(image_array[i, j] - image_array[i, j-1])
                c_w = np.exp(-(grad_w/kappa)**2)
                sum_of_neighbors += c_w * image_array[i, j-1]

            if j < shape[1]-1:
                grad_e = abs(image_array[i, j] - image_array[i, j+1])
                c_e = np.exp(-(grad_e/kappa)**2)
                sum_of_neighbors += c_e * image_array[i, j+1]

            diffused_value = image_array[i, j] + diffusion_coefficient * time_step * (sum_of_neighbors - 4 * image_array[i, j])
            diffused_image[i, j] = diffused_value

    return np.clip(diffused_image, 0, 255).astype(np.uint8)

# 使用非线性扩散函数替换原来的扩散函数
def denoise_image(image_path, diffusion_coefficient, time_step, iterations, kappa):
    image = Image.open(image_path).convert('L')
    image_array = np.array(image)
    denoised_image = np.copy(image_array)

    for _ in range(iterations):
        denoised_image = nonlinear_diffusion(denoised_image, diffusion_coefficient, time_step, kappa)

    return Image.fromarray(denoised_image, 'L')


In [48]:
diffused_image = diffuse_image('test/nii2png/11.png', 0.25, 10)
diffused_image.show()
diffused_image.save('test/nii2png/11_diffused.png')

In [54]:
denoised_image = denoise_image('test/nii2png/11_diffused.png', 0.5, 1, 10, 1)
denoised_image.show()
denoised_image.save('test/nii2png/11_denoised.png')

/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_25092/902279668.py:15: RuntimeWarning: overflow encountered in scalar subtract
  grad_s = abs(image_array[i, j] - image_array[i+1, j])
/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_25092/902279668.py:25: RuntimeWarning: overflow encountered in scalar subtract
  grad_e = abs(image_array[i, j] - image_array[i, j+1])
/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_25092/902279668.py:20: RuntimeWarning: overflow encountered in scalar subtract
  grad_w = abs(image_array[i, j] - image_array[i, j-1])
/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_25092/902279668.py:10: RuntimeWarning: overflow encountered in scalar subtract
  grad_n = abs(image_array[i, j] - image_array[i-1, j])
